# Building a Spam Filter with Naive Bayes Algorithim

In this project we will be using the multinomial naive Bayes algorithm to assign classify messages as either spam or non spam. The training dataset contains 5,572 SMS messages classifed as spam or non spam by humans. This data is courtesy of the [UCI Machine Leraning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection).

In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_csv('SMSSpamCollection', 
                            sep='\t', 
                            header=None, 
                            names=['Label', 'SMS'])
data.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
print(data.shape)

(5572, 2)


In [4]:
print('Percentage spam vs non spam (ham)')
data['Label'].value_counts() / data.shape[0] * 100

Percentage spam vs non spam (ham)


ham     86.593683
spam    13.406317
Name: Label, dtype: float64

## Separating training and test dataset

We will train the algorithm using 80% of the data and then test it against the 20% of the data remaining.

In [5]:
# Randomize data
random_data = data.sample(frac=1, random_state=1)

# Calc split index
split_index = round(len(data) * 0.8)

# Split training and test data
training_data = random_data[:split_index].reset_index(drop=True)
test_data = random_data[split_index:].reset_index(drop=True)

print('training data length: ', len(training_data))
print('test data length: ', len(test_data))

print('\nTraining data percent spam')
print(training_data['Label'].value_counts(normalize=True))
print('\nTest data percent spam')
print(test_data['Label'].value_counts(normalize=True))

training data length:  4458
test data length:  1114

Training data percent spam
ham     0.86541
spam    0.13459
Name: Label, dtype: float64

Test data percent spam
ham     0.868043
spam    0.131957
Name: Label, dtype: float64


Both sets of data have similar distributions of spam and non spam as the original dataset.

## Cleaning and Transforming Dataset

We will build a new dataframe that has a column for each unique word in the training_set's vocabulary. This will help us count the words we need to calculate the Bayes constants. The columns will be a count of the number of times a word appears in each SMS. The sum of the counts for each row will be equal to the number of words in the SMS.

In [6]:
training_data.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [7]:
# Clean and split the SMS messages into lists of words
training_data['SMS'] = training_data['SMS'].str.replace('\W', ' ')
training_data['SMS'] = training_data['SMS'].str.lower()
training_data['SMS'] = training_data['SMS'].str.split()

In [8]:
# Iterate over SMS column and create list of unique words in dataset
vocabulary = []
for message in training_data['SMS']:
    for word in message:
        vocabulary.append(word)
vocabulary = set(vocabulary)
vocabulary = list(vocabulary)
len(vocabulary)

7783

In [9]:
# Create a dictionary to store word counts for each SMS
word_counts_per_sms = ({unique_word: [0] * len(training_data['SMS'])
                        for unique_word in vocabulary}
                      )

for index, sms in enumerate(training_data['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

# Create new dataframe and join with original df to preserve labels
training_set = pd.DataFrame(word_counts_per_sms)
training_set = pd.concat([training_data, training_set], axis=1)

In [10]:
training_set.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Calculating Bayes Constants

- In order to calculate the probability of a word given spam or ham, we first need to calculate the constants $P(Spam)$ and $P(Ham)$ that will help us classify messages later in the project.


- We will then need to calculate $N_{Spam}$, $N_{Ham}$, and $N_{Vocabulary}$ found in the equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}


In [11]:
# Calculate constants needed for naive Bayes algo
alpha = 1
N_spam = (training_set[training_set['Label'] == 'spam']
          .sum(axis=1, numeric_only=True).sum()
         )

N_ham = (training_set[training_set['Label'] == 'ham']
          .sum(axis=1, numeric_only=True).sum()
         )

N_vocab = len(vocabulary)

P_spam = len(training_set[training_set['Label'] == 'spam']) / len(training_set)
P_ham = len(training_set[training_set['Label'] == 'ham']) / len(training_set)

## Calculating Bayes Parameters

We will use the following equations to calculate the parameters for the Bayes algoritim.

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

We have already calculated most of the terms in these equations above. We now need to calculate $N_{w_i|Spam}$ and $N_{w_i|ham}$.

1. We start by initializing two dictionaries that will contain all of the values of $P(w_i|Spam)$ and $P(w_i|ham)$.  
2. Next we create two dataframes, one for spam messages and one for ham messages. This will help us calculate the number of times a word appears in the set with respect to spam or ham.  
3. Iterate through the vocabulary and calculate the values of $P(w_i|Spam)$ and $P(w_i|ham).$ 

In [12]:
# Calculate parameters for spam and ham

p_words_given_spam = {}
p_words_given_ham = {}

training_set_spam = training_set[training_set['Label'] == 'spam']
training_set_ham = training_set[training_set['Label'] == 'ham']

for word in vocabulary:
    p_word_given_spam = ((training_set_spam[word].sum() + alpha) / 
                         (N_spam + alpha * N_vocab)
                        )
    p_word_given_ham = ((training_set_ham[word].sum() + alpha) / 
                        (N_ham + alpha * N_vocab)
                       )
    p_words_given_spam[word] = p_word_given_spam
    p_words_given_ham[word] = p_word_given_ham

## Defining Classification Function for Spam Filter

In order to classify a message as spam or not given the words in the message, we will need to calculate the following equations:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

We have everything we need to calculate these probabilities. We can write a spam filter as function that takes a message as an argument and classifies it as spam or ham. The function needs to:

- Clean up the message and split it into words
- Ignore words not in the vocabulary of the training set
- Iterate over the words in the message and calculate the probability that the message is spam or ham
- Compare the probabilities and classify the message corresponding to the larger probabiliy
- If probabilities are equal, return that the message needs human classification.

In [13]:
# Function to classify new message as spam or ham
def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()


    p_spam_given_message = P_spam
    p_ham_given_message = P_ham
    
    for word in message:
        if word in p_words_given_spam:
            p_spam_given_message *= p_words_given_spam[word]
        if word in p_words_given_ham:
            p_ham_given_message *= p_words_given_ham[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

## Inititial Test of Spam Filter

In [14]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [15]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Measuring the Spam Filter's Accuracy

We will modify the `classify` function above to classify the messages in our test data set.

In [16]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()


    p_spam_given_message = P_spam
    p_ham_given_message = P_ham
    
    for word in message:
        if word in p_words_given_spam:
            p_spam_given_message *= p_words_given_spam[word]
        if word in p_words_given_ham:
            p_ham_given_message *= p_words_given_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

We then apply the `classify_test_set` function to the test data set.

In [17]:
test_data['predicted'] = test_data['SMS'].apply(classify_test_set)
test_data.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


We can now measure the accuracy of the spam filter by calculating the following:

\begin{equation}
\text{Accuracy} = \frac{\text{number of correctly classified messages}}{\text{total number of classified messages}}
\end{equation}

We will do this by creating a new column that assigns 1 if the predicted value matches the actual value. We can then sum that column and divide by the length of the test set.

In [18]:
test_data['correct'] = 0
test_data.loc[test_data['Label'] == test_data['predicted'], 'correct'] = 1
test_data.head()

,Label,SMS,predicted,correct
0,ham,Later i guess. I needa do mcat study too.,ham,1
1,ham,But i haf enuff space got like 4 mb...,ham,1
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam,1
3,ham,All sounds good. Fingers . Makes it difficult ...,ham,1
4,ham,"All done, all handed in. Don't know if mega sh...",ham,1


In [23]:
correct = test_data['correct'].sum()
incorrect = len(test_data) - correct
accuracy = correct / len(test_data)
print('correct: ', correct, '\nincorrect: ', incorrect, '\naccuracy: ', accuracy)

correct:  1100 
incorrect:  14 
accuracy:  0.9874326750448833


## Results

The model was able to predict if a message was spam or not with a 98.74% accuracy.

Further work could investigate:
- Case sensitivity of messages